In [1]:
import geopandas as gpd
import pandas as pd

## Get County Data

In [2]:
file_path = "data/cb_2024_us_county_500k/cb_2024_us_county_500k.shp"
counties_gdf = gpd.read_file(file_path)

In [3]:
counties_gdf = counties_gdf.to_crs(9311)

## Open Denny's Data

In [4]:
dennys_gdf = gpd.read_file("data/dennys.gpkg")
dennys_gdf = dennys_gdf.to_crs(9311)

## Open IHOP Data

In [5]:
ihop_gdf = gpd.read_file("data/ihops.gpkg")
ihop_gdf = ihop_gdf.to_crs(9311)

## Open Perkins Data

In [6]:
perkins_gdf = gpd.read_file("data/perkins.gpkg")
perkins_gdf = perkins_gdf.to_crs(9311)

## Open Wafflehouse Data

In [7]:
wafflehouse_gdf = gpd.read_file("data/wafflehouse_locations.gpkg")
wafflehouse_gdf = wafflehouse_gdf.to_crs(9311)

## Intersect Data

In [8]:
def count_points_in_counties(points_gdf: gpd.GeoDataFrame, col_name):
    joined = gpd.sjoin(points_gdf, counties_gdf, how="left", predicate="within")
    counts = joined.groupby("index_right").size()
    result = counties_gdf.copy()
    result[col_name] = counts
    result[col_name] = result[col_name].fillna(0).astype(int)
    return result[[col_name]]

In [9]:
dennys_counts = count_points_in_counties(dennys_gdf, "DENNYs")
ihop_counts = count_points_in_counties(ihop_gdf, "IHOPs")
perkins_counts = count_points_in_counties(perkins_gdf, "PERKINs")
wh_counts = count_points_in_counties(wafflehouse_gdf, "WHs")

In [10]:
restaurant_counts_gdf = counties_gdf.copy()
restaurant_counts_gdf = (
    restaurant_counts_gdf.join(dennys_counts)
    .join(ihop_counts)
    .join(perkins_counts)
    .join(wh_counts)
)

In [11]:
rest_columns = ["DENNYs", "IHOPs", "PERKINs", "WHs"]

In [12]:
restaurant_counts_gdf[rest_columns] = (
    restaurant_counts_gdf[rest_columns].fillna(0).astype(int)
)

In [13]:
def get_tied_columns(row):
    max_val = row[rest_columns].max()
    if row["PERKINs"] == row["IHOPs"] == row["DENNYs"] == row["WHs"]:
        return "No Locations"
    return ", ".join(row[rest_columns].index[row[rest_columns] == max_val])

In [14]:
restaurant_counts_gdf["Label"] = restaurant_counts_gdf.apply(get_tied_columns, axis=1)

In [15]:
pd.DataFrame(restaurant_counts_gdf.groupby("Label").size()).reset_index().rename(
    columns={0: "COUNTS"}
).sort_values("COUNTS")

,Label,COUNTS
12,"PERKINs, WHs",2
8,"IHOPs, PERKINs, WHs",2
2,"DENNYs, IHOPs, PERKINs",4
3,"DENNYs, IHOPs, WHs",10
4,"DENNYs, PERKINs",15
7,"IHOPs, PERKINs",17
5,"DENNYs, WHs",21
9,"IHOPs, WHs",30
1,"DENNYs, IHOPs",69
11,PERKINs,97


In [16]:
restaurant_counts_gdf = restaurant_counts_gdf.to_crs(9311)

In [17]:
restaurant_counts_gdf.to_file("data/restaurant_counts.gpkg")